# Simple CNN Tutorial

This notebook presents a simple example of a 2D CNN use for classification. We choose to work with the Keras framework, as it is the most one of the most accessible ones. 
We propose to use cifar10 as a dataset. However, do not hesitate to use your own dataset instead.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
from keras.datasets import cifar10
import matplotlib.pyplot as plt

Load and prepare the data

In [ ]:
# input image dimensions
img_rows, img_cols = 32, 32

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
nb_classes = 10

# maximum number of samples
n_train = 10000
X_train = X_train[:n_train]
y_train = y_train[:n_train]

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

Plot a few images of the dataset

In [ ]:
from utils import draw_images
plt.figure(figsize=(15,15))
draw_images(X_train, 16,16);

### Example: A fully connected Neural Network
Here is how to implement a fully connected neural network implemented in Keras.
1. Using Keras, we implement a fully connected neural network using the following modules:
    * the `Sequential` model from `keras.models` to build a traditional feedforward model,
    * the `Dense` layer from `keras.layers` add a dense layer (linear),
    * and the `Dropout` layer from the same module to "regularize" our network.

2. Then we compile our model with a cross entropy loss (`keras.losses.categorical_crossentropy`). 
As on optimizer, a usual choice is ADAM (`keras.optimizers.Adam`)

3. We use the function `fit` to train your model.

4. Tensorboard is a very useful tool to detect and understand  problems in neural networks.


Code example
```
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LeakyReLU as lrelu

# 1) Create a model
model = Sequential()
# Add a fully connected layer with 32 outputs, 16 inputs and a relu activation
model.add(Dense(32,input_shape=[16], activation='relu'))
# Add a fully connected layer with 64 outputs, 32 inputs (this comes automatically from the previous layer) and no activation
model.add(Dense(64, activation=None))
# Add a leaky relu activation
model.add(lrelu(alpha=0.2))

# 2) compilation of the model. Here we choose the loss and the optimizer. The metrics is used to display the classification accuracy during training
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

# 3) Train the model. 
model.fit(X_train, l_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_valid, l_valid))
```

Reference for Keras:
* The official documentation: https://keras.io/
* If you already know neural networks, the official 'getting started' guide is good enough: https://keras.io/getting-started/sequential-model-guide/

# Implement a convolutional neural network to to classify cfar10

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

In [ ]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

Define some parameters

In [ ]:
batch_size = 32
nb_classes = 10
nb_epoch = 10

# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

input_shape = (img_rows, img_cols, 3)

Build the CNN. 

In [ ]:
model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size,
                        padding='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size, padding='valid'))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size, padding='valid'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

Now we need to compile our model

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Show a summary of the model parameters. This is important as it can tell which part of the net use memory or has more modelization power.

In [ ]:
model.summary()

In [ ]:
# Tensorboard
from keras.callbacks import TensorBoard
import os
output_path = 'tensorboard'
tfdir = os.path.join(output_path)
os.makedirs(tfdir, exist_ok=True)
tbCallBack = TensorBoard(log_dir=tfdir, histogram_freq=0, write_graph=True, write_images=True, update_freq=10000)


Now it is time to train the network. Increase the number of samples to get better results. A GPU will help signigicantly as well.

In [ ]:
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test), callbacks=[tbCallBack])

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# The test accuracy is highter during the first epochs because of the dropout... 